In [1]:
import run_simulation as rs
import numpy as np	
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import tdgl
import os 
rs.check_system()

GPU is available
GPU:  b'NVIDIA GeForce RTX 4060 Laptop GPU'


c:\Program Files\Python310\lib\site-packages\cupyx\jit\_interface.py:173: FutureWarning: cupyx.jit.rawkernel is experimental. The interface can change in the future.
  cupy._util.experimental('cupyx.jit.rawkernel')


In [2]:
folder_path = 'Ls_4'

xi=1

min_L=0.4
max_L=20
num_L=10

min_c=0
max_c=20
num_c=100

L_range=np.round(np.linspace(min_L,max_L,num_L),2)
current_range=np.round(np.linspace(min_c,max_c,num_c), 3)


In [3]:


files = rs.get_all_files(folder_path)

data_list=[]

for file in files:
	L, current = rs.extract_numbers(file)
	array=np.load(file)
	data=[L,current,array]
	data_list.append(data)

In [4]:
Ls_list=[]
currents_for_Ls=[]
currents=[]

for data in data_list:
	if data[0] not in Ls_list:
		Ls_list.append(data[0])
		currents_for_Ls.append(currents)
		currents=[data[1]]
	else:
		currents.append(data[1])

currents_for_Ls.append(currents)
currents_for_Ls=currents_for_Ls[1:]

found_Ls,missing_Ls=rs.check_elements_in_list(L_range,Ls_list)

print(f"found data for L= {found_Ls}")
print(f"missing data for L= {missing_Ls}")

simulations_to_be_run={}

for ii in range(len(found_Ls)):
	found_c,missing_c=rs.check_elements_in_list(current_range,currents_for_Ls[ii])

	simulations_to_be_run[found_Ls[ii]]=np.array(missing_c)
	
for ii in range(len(missing_Ls)):
	simulations_to_be_run[missing_Ls[ii]]=current_range


for key, value in simulations_to_be_run.items():
    print(f"{key}: {value}")



found data for L= [0.4, 2.58, 4.76, 6.93, 9.11, 11.29]
missing data for L= [13.47, 15.64, 17.82, 20.0]
0.4: []
2.58: []
4.76: []
6.93: []
9.11: []
11.29: []
13.47: [ 0.     0.202  0.404  0.606  0.808  1.01   1.212  1.414  1.616  1.818
  2.02   2.222  2.424  2.626  2.828  3.03   3.232  3.434  3.636  3.838
  4.04   4.242  4.444  4.646  4.848  5.051  5.253  5.455  5.657  5.859
  6.061  6.263  6.465  6.667  6.869  7.071  7.273  7.475  7.677  7.879
  8.081  8.283  8.485  8.687  8.889  9.091  9.293  9.495  9.697  9.899
 10.101 10.303 10.505 10.707 10.909 11.111 11.313 11.515 11.717 11.919
 12.121 12.323 12.525 12.727 12.929 13.131 13.333 13.535 13.737 13.939
 14.141 14.343 14.545 14.747 14.949 15.152 15.354 15.556 15.758 15.96
 16.162 16.364 16.566 16.768 16.97  17.172 17.374 17.576 17.778 17.98
 18.182 18.384 18.586 18.788 18.99  19.192 19.394 19.596 19.798 20.   ]
15.64: [ 0.     0.202  0.404  0.606  0.808  1.01   1.212  1.414  1.616  1.818
  2.02   2.222  2.424  2.626  2.828  3.03   3.232

In [ ]:

for L in simulations_to_be_run:

	currents=simulations_to_be_run[L]
	# print(currents.shape, currents.size == 0)
	if currents.size == 0:
		continue
	print(f"Running simulations for L={L} with currents: {currents}")
	device=rs.define_4_terminal_mesh(L,L,xi=xi,
								  probes=[((-L*(0.4),0),
					   				(L*(0.4),0),)],
									terminal_width=L/30,
									terminal_height=L/30,

								  )
	rs.build_mesh(device,max_edge_L=xi/4,plot=False)

	solutions,currents_out=rs.current_series(device,
									  current_range=currents,
									  H_field=10,
									  solve_time=300
									  )
	
	jj=0
	for solution in solutions:
		voltage=solution.dynamics.voltage()
		time=solution.dynamics.time
		solution.to_hdf5(f'{folder_path}_solutions/solution_{L}_{currents_out[jj]}_.hdf5')

		np.save(f'{folder_path}/voltage_{L}_{currents_out[jj]}_.npy',np.array([time,voltage]))

		jj+=1

Running simulations for L=13.47 with currents: [ 0.     0.202  0.404  0.606  0.808  1.01   1.212  1.414  1.616  1.818
  2.02   2.222  2.424  2.626  2.828  3.03   3.232  3.434  3.636  3.838
  4.04   4.242  4.444  4.646  4.848  5.051  5.253  5.455  5.657  5.859
  6.061  6.263  6.465  6.667  6.869  7.071  7.273  7.475  7.677  7.879
  8.081  8.283  8.485  8.687  8.889  9.091  9.293  9.495  9.697  9.899
 10.101 10.303 10.505 10.707 10.909 11.111 11.313 11.515 11.717 11.919
 12.121 12.323 12.525 12.727 12.929 13.131 13.333 13.535 13.737 13.939
 14.141 14.343 14.545 14.747 14.949 15.152 15.354 15.556 15.758 15.96
 16.162 16.364 16.566 16.768 16.97  17.172 17.374 17.576 17.778 17.98
 18.182 18.384 18.586 18.788 18.99  19.192 19.394 19.596 19.798 20.   ]


Constructing Voronoi polygons: 100%|██████████| 9653/9653 [00:07<00:00, 1283.61it/s]


num_sites: 9653
num_elements: 18904
min_edge_length: 0.06735039753051353
max_edge_length: 0.24599733801444051
mean_edge_length: 0.149334221436184
min_area: 0.0038967481224259515
max_area: 0.03772287490284893
mean_area: 0.018796322386822748
coherence_length: 0.1
length_units: um


Simulating: 100%|█████████▉| 300/300 [02:52<00:00,  1.74tau/s ]


1/100 done


...

Simulating: 100%|█████████▉| 300/300 [07:01<00:00,  1.40s/tau ]    


41/100 done


Simulating: 100%|█████████▉| 300/300 [07:05<00:00,  1.42s/tau ]    


42/100 done


Simulating:  69%|██████▊   | 206/300 [05:19<01:28,  1.06tau/s ]    